# Fed-BioMed Researcher base example

Use for developing (autoreloads changes made across packages)

In [1]:
%load_ext autoreload
%autoreload 2

## Start the network
Before running this notebook, start the network with `./scripts/fedbiomed_run network`

## Setting the node up
It is necessary to previously configure a node:
1. `./scripts/fedbiomed_run node add`
  * Select option 2 (default) to add MNIST to the node
  * Confirm default tags by hitting "y" and ENTER
  * Pick the folder where MNIST is downloaded (this is due torch issue https://github.com/pytorch/vision/issues/3549)
  * Data must have been added (if you get a warning saying that data must be unique is because it's been already added)
  
2. Check that your data has been added by executing `./scripts/fedbiomed_run node list`
3. Run the node using `./scripts/fedbiomed_run node run`. Wait until you get `Starting task manager`. it means you are online.

## Define an experiment model and parameters"

Declare a torch.nn MyTrainingPlan class to send for training on the node

In [2]:
import torch
import torch.nn as nn
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from torchvision import datasets, transforms

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(MyTrainingPlan, self).__init__(model_args)
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["from torchvision import datasets, transforms"]
        
        self.add_dependency(deps)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        
        
        output = F.log_softmax(x, dim=1)
        return output

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        return DataManager(dataset=dataset1, **train_kwargs)
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss


This group of arguments correspond respectively:
* `model_args`: a dictionary with the arguments related to the model (e.g. number of layers, features, etc.). This will be passed to the model class on the node side.
* `training_args`: a dictionary containing the arguments for the training routine (e.g. batch size, learning rate, epochs, etc.). This will be passed to the routine on the node side.

**NOTE:** typos and/or lack of positional (required) arguments will raise error. 🤓

In [3]:
model_args = {}

training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 100 # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
}

## Declare and run the experiment

- search nodes serving data for these `tags`, optionally filter on a list of node ID with `nodes`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `round_limit` rounds, applying the `node_selection_strategy` between the rounds

In [4]:
from fedbiomed.researcher.experiment_jl import ExperimentJL
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 2

exp = ExperimentJL(tags=tags,
                 model_args=model_args,
                 model_class=MyTrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

2022-09-02 09:12:01,376 fedbiomed INFO - Component environment:
2022-09-02 09:12:01,377 fedbiomed INFO - type = ComponentType.RESEARCHER
2022-09-02 09:12:01,459 fedbiomed INFO - Messaging researcher_f07a5365-9039-453c-94c8-41b2ad3a2c8c successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x7f5b52429250>
2022-09-02 09:12:01,469 fedbiomed INFO - Searching dataset with data tags: ['#MNIST', '#dataset'] for all nodes
2022-09-02 09:12:11,483 fedbiomed INFO - Node selected for training -> node_ba731923-352e-49eb-bd39-a95937842cdb
2022-09-02 09:12:11,485 fedbiomed INFO - Node selected for training -> node_b7843934-7037-4ec0-809f-41538850d760
2022-09-02 09:12:11,489 fedbiomed INFO - Checking data quality of federated datasets...
2022-09-02 09:12:11,505 fedbiomed DEBUG - Model file has been saved: /workspaces/PycharmProjects/fedbiomed/var/experiments/Experiment_0002/my_model_24fea261-6e69-447f-934c-4dab48e74bdd.py
2022-09-02 09:12:11,529 fedbiom

Let's start the experiment.

By default, this function doesn't stop until all the `round_limit` rounds are done for all the nodes

In [ ]:
exp.run_once(increase=True)

In [5]:
exp.run(rounds=rounds, increase=True)

2022-09-02 09:12:20,415 fedbiomed INFO - Sampled nodes in round 0 ['node_ba731923-352e-49eb-bd39-a95937842cdb', 'node_b7843934-7037-4ec0-809f-41538850d760']
2022-09-02 09:12:20,416 fedbiomed INFO - Sending request 
					 To: node_ba731923-352e-49eb-bd39-a95937842cdb 
					 Request: : Perform training with the arguments: {'researcher_id': 'researcher_f07a5365-9039-453c-94c8-41b2ad3a2c8c', 'job_id': '2cc861a8-8282-4f4a-8817-d3e656541289', 'current_round': 0, 'training_args': scheme:
{'lr': {'rules': [<class 'float'>, <function TrainingArgs._lr_hook at 0x7f5b5255d3a0>], 'required': False}, 'batch_size': {'rules': [<class 'int'>], 'required': False}, 'epochs': {'rules': [<class 'int'>], 'required': False}, 'dry_run': {'rules': [<class 'bool'>], 'required': False}, 'batch_maxnum': {'rules': [<class 'int'>], 'required': False}, 'test_ratio': {'rules': [<class 'float'>, <function TrainingArgs._test_ratio_hook at 0x7f5b5255d280>], 'required': False, 'default': 0.0}, 'test_on_local_updates': {'

2022-09-02 09:12:25,758 fedbiomed INFO - TRAINING 
					 NODE_ID: node_ba731923-352e-49eb-bd39-a95937842cdb 
					 Epoch: 1 | Completed: 3840/60000 (6%) 
 					 Loss: 0.212863 
					 ---------
2022-09-02 09:12:26,335 fedbiomed INFO - TRAINING 
					 NODE_ID: node_b7843934-7037-4ec0-809f-41538850d760 
					 Epoch: 1 | Completed: 4320/60000 (7%) 
 					 Loss: 0.185636 
					 ---------
2022-09-02 09:12:26,423 fedbiomed INFO - TRAINING 
					 NODE_ID: node_ba731923-352e-49eb-bd39-a95937842cdb 
					 Epoch: 1 | Completed: 4320/60000 (7%) 
 					 Loss: 0.315479 
					 ---------
2022-09-02 09:12:26,959 fedbiomed INFO - INFO
					 NODE node_b7843934-7037-4ec0-809f-41538850d760
					 MESSAGE: Reached 100 batches for this epoch, ignore remaining data
-----------------------------------------------------------------
2022-09-02 09:12:27,019 fedbiomed INFO - INFO
					 NODE node_b7843934-7037-4ec0-809f-41538850d760
					 MESSAGE: Quantization of model parameters
------------------------------------

2022-09-02 09:12:36,732 fedbiomed DEBUG - researcher_f07a5365-9039-453c-94c8-41b2ad3a2c8c
2022-09-02 09:12:36,764 fedbiomed INFO - WARNING
					 NODE node_ba731923-352e-49eb-bd39-a95937842cdb
					 MESSAGE: There is no validation activated for the round. Please set flag for `test_on_global_updates`, `test_on_local_updates`, or both. Splitting dataset for validation will be ignored
-----------------------------------------------------------------
2022-09-02 09:12:36,786 fedbiomed INFO - INFO
					 NODE node_ba731923-352e-49eb-bd39-a95937842cdb
					 MESSAGE: training with arguments {'history_monitor': <fedbiomed.node.history_monitor.HistoryMonitor object at 0x7f5ba4e22100>, 'node_args': {'gpu': False, 'gpu_num': None, 'gpu_only': False}, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100}
-----------------------------------------------------------------
2022-09-02 09:12:36,827 fedbiomed INFO - WARNING
					 NODE node_b7843934-7037-4ec0-809f-41538850d760
					 MESSAGE: There

2022-09-02 09:12:45,522 fedbiomed INFO - INFO
					 NODE node_ba731923-352e-49eb-bd39-a95937842cdb
					 MESSAGE: Protecting model parameters done, length: 17646
-----------------------------------------------------------------
2022-09-02 09:12:45,527 fedbiomed INFO - INFO
					 NODE node_b7843934-7037-4ec0-809f-41538850d760
					 MESSAGE: Protecting model parameters done, length: 17646
-----------------------------------------------------------------
2022-09-02 09:12:45,678 fedbiomed INFO - INFO
					 NODE node_ba731923-352e-49eb-bd39-a95937842cdb
					 MESSAGE: results uploaded successfully 
-----------------------------------------------------------------
2022-09-02 09:12:45,690 fedbiomed INFO - INFO
					 NODE node_b7843934-7037-4ec0-809f-41538850d760
					 MESSAGE: results uploaded successfully 
-----------------------------------------------------------------
2022-09-02 09:12:51,749 fedbiomed INFO - Downloading model params after training on node_ba731923-352e-49eb-bd39-a95937842

2

Local training results for each round and each node are available via `exp.training_replies()` (index 0 to (`rounds` - 1) ).

For example you can view the training results for the last round below.

Different timings (in seconds) are reported for each dataset of a node participating in a round :
- `rtime_training` real time (clock time) spent in the training function on the node
- `ptime_training` process time (user and system CPU) spent in the training function on the node
- `rtime_total` real time (clock time) spent in the researcher between sending the request and handling the response, at the `Job()` layer

In [ ]:
print("\nList the training rounds : ", exp.training_replies().keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies()[rounds - 1].data()
for c in range(len(round_data)):
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = round_data[c]['node_id'],
        rtraining = round_data[c]['timing']['rtime_training'],
        ptraining = round_data[c]['timing']['ptime_training'],
        rtotal = round_data[c]['timing']['rtime_total']))
print('\n')
    
exp.training_replies()[rounds - 1].dataframe()

Federated parameters for each round are available via `exp.aggregated_params()` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [ ]:
print("\nList the training rounds : ", exp.aggregated_params().keys())

print("\nAccess the federated params for the last training round :")
print("\t- params_path: ", exp.aggregated_params()[rounds - 1]['params_path'])
print("\t- parameter data: ", exp.aggregated_params()[rounds - 1]['params'].keys())


Feel free to run other sample notebooks or try your own models :D